In [1]:
import pandas as pd
from datetime import datetime,timedelta
import numpy as np
import math
from scipy.spatial import cKDTree
import datetime as dt
import glob

%matplotlib inline

In [2]:
def read_all(path:str):
#  glob.glob(pathname, *, root_dir=None, dir_fd=None, recursive=False, include_hidden=False)
    filename = glob.glob(path)
    df = pd.read_csv(filename[0])
    print(f'{filename[0]} has been read')
    for index, i in enumerate(filename):
        if index == 0:
            continue
        else:
            df = pd.concat([df, pd.read_csv(i)], ignore_index=True)
        print(f'{i} has been read')
    return df
        

In [3]:
data_tap_in_gate = read_all('9. September/IN/*.csv')
data_tap_out_gate = read_all('9. September/OUT/*.csv')
# data_tap_in_tob = read_all('OD TOB OKTOBER 2022/IN/*.csv')
# data_tap_out_tob = read_all('OD TOB OKTOBER 2022/OUT/*.csv')

9. September/IN\20231009 Data Tap IN Gate 1 - 10 September 2023.csv has been read
9. September/IN\20231009 Data Tap IN Gate 11 - 20 September 2023.csv has been read
9. September/IN\20231009 Data Tap IN Gate 21 - 30 September 2023.csv has been read
9. September/OUT\20231009 Data Tap OUT Gate 1 - 10 September 2023.csv has been read
9. September/OUT\20231009 Data Tap OUT Gate 11 - 20 September 2023.csv has been read
9. September/OUT\20231009 Data Tap OUT Gate 21- 30 September 2023.csv has been read


In [4]:
data_tap_in_gate.shape

(10512624, 7)

data_tap_in_tob.shape

In [5]:
data_tap_out_gate.shape

(10082255, 7)

data_tap_out_tob.shape

data_tap_out_tob.info()

# Read Data

In [ ]:
data_tap_in_gate = pd.read_csv('OD GATE TOB Agustus 2023/gate_for_OD_IN Agustus 2023.csv')
data_tap_out_gate = pd.read_csv('OD GATE TOB Agustus 2023/gate_for_OD_IN Agustus 2023.csv')
data_tap_in_tob = pd.read_csv('bahan od nov 2022/In TOB Nov 2022.csv')
data_tap_out_tob = pd.read_csv('bahan od nov 2022/Out TOB Nov 2022.csv')

data_tap_in_gate.info()

data_tap_in_tob.info()

# Menambahkan data Bus Stop pada transaksi TOB

### Read Data Bus stop

In [6]:
data_bus_stop = pd.read_csv('tj_stops.csv')

In [7]:
data_bus_stop['bus_stop'] = data_bus_stop.index

In [8]:
data_bus_stop.head()

,Stop ID,Nama Perhentian Bus,Koordinat,Tipe Perhentian Bus,Tipe Halte,Kecamatan,Kota,Latitude,Longitude,Jumlah Rute,Rute yang Melewati,Stop ID (2),Koridor - Pelayanan,Jenis Perhentian Bus,Tipe Penanda,bus_stop
0,B00001P,18 Office Park,"-6.299146, 106.832099",Bus Stop,-,Pasar Minggu,Jakarta Selatan,-6.299146,106.832099,1,JAK.46,B00001P,Non-koridor,Particular,NaN,0
1,B00002P,ABA,"-6.194149, 106.839385",Bus Stop,-,Menteng,Jakarta Pusat,-6.194149,106.839385,4,"5M, 6H, JAK.10A, JAK.10B",B00002P,Non-koridor,Particular,NaN,1
2,B00003P,Acacia Residence,"-6.263358, 106.756754",Bus Stop,-,Pesanggrahan,Jakarta Selatan,-6.263358,106.756754,1,8E,B00003P,Non-koridor,Particular,NaN,2
3,B00004P,ACC Simatupang,"-6.304475, 106.84858",Bus Stop,-,Jagakarsa,Jakarta Selatan,-6.304475,106.848580,3,"7A, 7E, D21",B00004P,Non-koridor,Particular,NaN,3
4,B00005P,ACE Hardware,"-6.387532, 106.827383",Bus Stop,-,Beji,Depok,-6.387532,106.827383,1,D11,B00005P,Non-koridor,Particular,NaN,4


In [9]:
data_bus_stop.drop(['Stop ID','Koordinat','Tipe Perhentian Bus','Tipe Halte','Stop ID (2)','Tipe Penanda'],axis=1,inplace=True)

### Menambahkan data bus stop pada data tap in tob

In [ ]:
# r = 1 degree ~~ 111km, r = 0.00045 ~~ 50m
# r = 1 degree ~~ 111km, r = 0.0009 ~~ 100m

ck = cKDTree(data_bus_stop[['Latitude','Longitude']].values)

dataset3 = ck.query_ball_point(data_tap_in_tob[['p_latitude_flo','p_longitude_flo']].values, r=0.00045, p=2)

In [ ]:
y=[]
for i in range (len(dataset3)):
    try:
        y.append(dataset3[i][0])
    except:
        y.append(np.nan)

In [ ]:
data_tap_in_tob['bus_stop'] = y

In [ ]:
data_tap_in_tob = data_tap_in_tob.merge(data_bus_stop[['Nama Perhentian Bus','bus_stop','Latitude','Longitude','Kecamatan','Kota']],how = 'left', on='bus_stop')

In [ ]:
data_tap_in_tob.drop(['shelter_name_var','bus_stop','p_latitude_flo','p_longitude_flo'],axis=1,inplace=True)

In [ ]:
data_tap_in_tob.rename({'Nama Perhentian Bus':'shelter_name_var'},axis=1,inplace=True)

In [ ]:
data_tap_in_tob['layanan'] = 'TOB'

In [ ]:
data_tap_in_tob.head()

### Menambahkan data bus stop pada data tap out tob

In [ ]:
# r = 1 degree ~~ 111km, r = 0.00045 ~~ 50m
# r = 1 degree ~~ 111km, r = 0.0009 ~~ 100m

ck = cKDTree(data_bus_stop[['Latitude','Longitude']].values)

dataset3 = ck.query_ball_point(data_tap_out_tob[['p_latitude_flo','p_longitude_flo']].values, r=0.00045, p=2)

In [ ]:
y=[]
for i in range (len(dataset3)):
    try:
        y.append(dataset3[i][0])
    except:
        y.append(np.nan)

In [ ]:
data_tap_out_tob['bus_stop'] = y

In [ ]:
data_tap_out_tob = data_tap_out_tob.merge(data_bus_stop[['Nama Perhentian Bus','bus_stop','Latitude','Longitude','Kecamatan','Kota']],how = 'left', on='bus_stop')

In [ ]:
data_tap_out_tob.drop(['shelter_name_var','bus_stop','p_latitude_flo','p_longitude_flo'],axis=1,inplace=True)

In [ ]:
data_tap_out_tob.rename({'Nama Perhentian Bus':'shelter_name_var'},axis=1,inplace=True)

In [ ]:
data_tap_out_tob['layanan'] = 'TOB'

In [ ]:
data_tap_out_tob.head()

# GET LAT, LON, Kecamatan & Kota for Gate Transaction

In [10]:
# data_tap_in_gate = data_tap_in_gate.merge(data_bus_stop[['Nama Perhentian Bus','Latitude','Longitude','Kecamatan','Kota']],how = 'left', left_on=data_tap_in_gate['shelter_name_var'].str.lower(), right_on=data_bus_stop['Nama Perhentian Bus'].str.lower())
# data_tap_out_gate = data_tap_out_gate.merge(data_bus_stop[['Nama Perhentian Bus','Latitude','Longitude','Kecamatan','Kota']],how = 'left', left_on=data_tap_out_gate['shelter_name_var'].str.lower(), right_on=data_bus_stop['Nama Perhentian Bus'].str.lower())
data_tap_in_gate = data_tap_in_gate.merge(data_bus_stop[['Latitude','Longitude','Kecamatan','Kota']],how = 'left', left_on=data_tap_in_gate['shelter_name_var'].str.lower(), right_on=data_bus_stop['Nama Perhentian Bus'].str.lower())
data_tap_out_gate = data_tap_out_gate.merge(data_bus_stop[['Latitude','Longitude','Kecamatan','Kota']],how = 'left', left_on=data_tap_out_gate['shelter_name_var'].str.lower(), right_on=data_bus_stop['Nama Perhentian Bus'].str.lower())
# data_tap_in_gate.drop(['shelter_name_var','key_0'], axis=1, inplace=True)
# data_tap_out_gate.drop(['shelter_name_var','key_0'], axis=1, inplace=True)
# data_tap_in_gate.rename({'Nama Perhentian Bus':'shelter_name_var'},axis=1,inplace=True)
# data_tap_out_gate.rename({'Nama Perhentian Bus':'shelter_name_var'},axis=1,inplace=True)

In [ ]:
data_tap_out_gate.head()

# Union data TOB dan GATE

In [11]:
data_tap_in_gate['layanan']='Gate'
data_tap_out_gate['layanan']='Gate'

data_tap_out_tob.loc[data_tap_out_tob['shelter_name_var'].isnull(), 'shelter_name_var'] = 'not bus stop'
data_tap_in_tob.loc[data_tap_in_tob['shelter_name_var'].isnull(), 'shelter_name_var'] = 'not bus stop'

data_tap_in = pd.concat([data_tap_in_gate,data_tap_in_tob], ignore_index=True)
data_tap_out = pd.concat([data_tap_out_gate,data_tap_out_tob], ignore_index=True)

In [12]:
data_tap_in = data_tap_in_gate.copy()
data_tap_out = data_tap_out_gate.copy()

In [13]:
data_tap_in.shape

(10512624, 13)

In [14]:
data_tap_out.shape

(10082255, 13)

In [15]:
len(data_tap_in)+len(data_tap_out)

20594879

# Matching data Tap in dan Tap out

In [16]:
data_tap_in['waktu_tap'] = data_tap_in['waktu_tap'].astype('datetime64[ns]')
data_tap_out['waktu_tap'] = data_tap_out['waktu_tap'].astype('datetime64[ns]')
data_tap_in['tanggal'] = data_tap_in['waktu_tap'].dt.date
data_tap_out['tanggal'] = data_tap_out['waktu_tap'].dt.date

In [17]:
data_tap_out['waktu_out'] = data_tap_out['waktu_tap']
data_tap_in['waktu_in'] = data_tap_in['waktu_tap']

In [18]:
data_tap_in['card_number_var'] = data_tap_in['card_number_var'].astype('str')
data_tap_out['card_number_var'] = data_tap_out['card_number_var'].astype('str')

In [19]:
data_OD2 = pd.merge_asof(data_tap_in.sort_values('waktu_tap'),data_tap_out.sort_values('waktu_tap'),on='waktu_tap',by=['tanggal','card_number_var'],direction='forward',suffixes=('_in','_out'),tolerance=pd.Timedelta("3hours"),allow_exact_matches=False,)

left_merge = pd.merge_asof(data_tap_in.sort_values('waktu_tap'), data_tap_out.sort_values('waktu_tap'),
    tolerance=pd.Timedelta('3hours'), direction='forward', 
    on='waktu_tap', 
    by=['tanggal','card_number_var'],
    suffixes=('_in','_out'))

right_merge = pd.merge_asof(data_tap_out.sort_values('waktu_tap'), data_tap_in.sort_values('waktu_tap'), 
    tolerance=pd.Timedelta('3hours'), direction='backward', 
    on='waktu_tap',
    by=['tanggal','card_number_var'],
    suffixes=('_out','_in'))

data_OD2 = (left_merge.merge(right_merge, how="outer")
    .sort_values(['waktu_tap'])
    .reset_index(drop=True))

data_OD2['shelter_name_var_in'] = np.where(data_OD2['waktu_tap'] == data_OD2['waktu_in'],data_OD2['shelter_name_var_in'],np.nan)
data_OD2['layanan_in'] = np.where(data_OD2['waktu_tap'] == data_OD2['waktu_in'],data_OD2['layanan_in'],np.nan)
data_OD2['shelter_name_var_out'] = data_OD2['shelter_name_var_out'].mask(data_OD2.duplicated(['transcode_txt_out']))
data_OD2['layanan_out'] = data_OD2['layanan_out'].mask(data_OD2.duplicated(['transcode_txt_out']))

In [20]:
data_OD2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10512624 entries, 0 to 10512623
Data columns (total 27 columns):
 #   Column                Dtype         
---  ------                -----         
 0   key_0_in              object        
 1   shelter_name_var_in   object        
 2   shelter_code_in       int64         
 3   terminal_code_in      int64         
 4   waktu_tap             datetime64[ns]
 5   card_number_var       object        
 6   tap_in                object        
 7   transcode_txt_in      object        
 8   Latitude_in           float64       
 9   Longitude_in          float64       
 10  Kecamatan_in          object        
 11  Kota_in               object        
 12  layanan_in            object        
 13  tanggal               object        
 14  waktu_in              datetime64[ns]
 15  key_0_out             object        
 16  shelter_name_var_out  object        
 17  shelter_code_out      float64       
 18  terminal_code_out     float64       
 19

In [21]:
# data_OD2.drop(['shelter_code_in','terminal_code_in','waktu_tap','card_number_var','tap_in','tanggal','shelter_code_out','terminal_code_out','tap_out'],axis=1,inplace=True)
data_OD2.drop(['shelter_code_in','terminal_code_in','tap_in','waktu_tap','tanggal','shelter_code_out','terminal_code_out','tap_out'],axis=1,inplace=True)

In [22]:
# data_OD2 = data_OD2[data_OD2['Kecamatan_in'].notnull() & data_OD2['Kecamatan_out'].notnull()]

data_OD2.to_csv('Result_OD_BRT_NBRT_Transjakarta_Okt_2022 rev1.csv')

In [23]:
data_OD2.to_csv('Result_OD_BRT_Transjakarta_Sep_2023 rev1.csv')

In [24]:
data_OD2

,key_0_in,shelter_name_var_in,card_number_var,transcode_txt_in,Latitude_in,Longitude_in,Kecamatan_in,Kota_in,layanan_in,waktu_in,key_0_out,shelter_name_var_out,transcode_txt_out,Latitude_out,Longitude_out,Kecamatan_out,Kota_out,layanan_out,waktu_out
0,jelambar,JELAMBAR,145200004000198,NaN,-6.166548,106.786505,Grogol Petamburan,Jakarta Barat,Gate,2023-09-01 00:00:07,pulo gadung,PULO GADUNG,0145200004000198280833202309010140440000009100...,-6.183245,106.908683,Pulo Gadung,Jakarta Timur,Gate,2023-09-01 01:40:43
1,tegal parang arah p.ranti,TEGAL PARANG ARAH P.RANTI,6032984013611979,NaN,NaN,NaN,NaN,NaN,Gate,2023-09-01 00:00:10,pinang ranti,PINANG RANTI,603298401361197900030D706E857D9963392104000120...,-6.291075,106.886338,Makasar,Jakarta Timur,Gate,2023-09-01 00:33:48
2,bnn,BNN,6032984000908966,NaN,-6.246510,106.873180,Jatinegara,Jakarta Timur,Gate,2023-09-01 00:00:14,manggarai,MANGGARAI,603298400090896600020D706E857E5EC7340402000120...,-6.208880,106.847391,Setiabudi,Jakarta Selatan,Gate,2023-09-01 00:51:43
3,pemuda rw.mangun,PEMUDA RW.MANGUN,145015300049719,NaN,NaN,NaN,NaN,NaN,Gate,2023-09-01 00:00:16,walikota jakarta timur,WALIKOTA JAKARTA TIMUR,0145015300049719280833202309010131050000007500...,-6.212540,106.945367,Cakung,Jakarta Timur,Gate,2023-09-01 01:31:04
4,jelambar,JELAMBAR,6032982838756631,NaN,-6.166548,106.786505,Grogol Petamburan,Jakarta Barat,Gate,2023-09-01 00:00:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10512619,masjid agung,MASJID AGUNG,145200310750874,NaN,-6.236483,106.798447,Kebayoran Baru,Jakarta Selatan,Gate,2023-09-30 23:59:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10512620,ancol,ANCOL,6032984070849421,NaN,-6.127563,106.830391,Pademangan,Jakarta Utara,Gate,2023-09-30 23:59:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10512621,jembatan gantung,JEMBATAN GANTUNG,145015300095472,NaN,-6.155513,106.749464,Cengkareng,Jakarta Barat,Gate,2023-09-30 23:59:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10512622,bundaran hi,BUNDARAN HI,1003012206990513,NaN,-6.193021,106.822997,Menteng,Jakarta Pusat,Gate,2023-09-30 23:59:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [ ]:
data_tap_in.to_csv('data_tap_in_BRT-NonBRT 6-12 Des.csv')

In [ ]:
data_tap_out.to_csv('data_tap_out_BRT-NonBRT 6-12 Des.csv')

In [ ]:
data_OD2[data_OD2['transcode_txt_out'].isnull()]

In [ ]:
data_OD2[data_OD2['card_number_var']=='7546260000552188']

In [ ]:
len(data_OD2[data_OD2['card_number_var']=='7546270008492188'])

In [ ]:
data_tap_in[data_tap_in['card_number_var']=='7546260000552188']

In [ ]:
len(data_tap_in[data_tap_in['card_number_var']=='7546270008492188'])

In [ ]:
data_tap_out[data_tap_out['card_number_var']=='7546270008492188']

In [ ]:
len(data_tap_out[data_tap_out['card_number_var']=='7546270008492188'])

data_OD2[data_OD2['card_number_var']=='6032984003020660']

data_tap_in[data_tap_in['card_number_var']=='6032984003020660']

data_tap_out[data_tap_out['card_number_var']=='6032984003020660']